In [ ]:
import xlrd
import pandas as pd
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
from snowflake.snowpark.files import SnowflakeFile
from datetime import datetime
import sys
sys.path.append('..')
from credentials import Credentials

In [ ]:
cred = Credentials()
session = Session.builder.configs(cred.__dict__).create()

In [ ]:
session.use_role("SYSADMIN")
session.use_database("HACKATHON")
session.use_warehouse("HACKATHON_VW")
session.use_schema("PUBLIC")

In [ ]:
class FileFormat:
	def __init__(self, order_date:str, allocation_date:str, so_number:str, so_line:str, cust_po:str, end_user_po:str, account_rep:str, p_line:str, td_pn:str, manuf_pn:str):
		self.order_date = order_date
		self.allocation_date = allocation_date
		self.so_number = so_number
		self.so_line = so_line
		self.cust_po = cust_po
		self.end_user_po = end_user_po
		self.account_rep = account_rep
		self.p_line = p_line
		self.td_pn = td_pn

	@staticmethod
	def keys():
		return ['order_date', 'allocation_date', 'so_number', 'so_line', 'cust_po', 'end_user_po', 'account_rep', 'p_line', 'td_pn', 'manuf_pn']

	def values(self):
		return self.__dict__
	
	def __del__(self):
		"This (Magic/Dunder) method deletes the object from memory"
		pass

In [ ]:
def read_first_workbook_from_excel(session: Session, file_path: str):
    with open(file_path, 'rb') as file:
        binary_data = file.read()
    
    workbook = xlrd.open_workbook(file_contents=binary_data, on_demand=True)
    
    worksheet = workbook.sheet_by_index(0)

    data = []
    
    for row in range(1, worksheet.nrows):

        order_date = worksheet.cell(row, 0).value
        if order_date == '':
            order_date = None
        else:
            order_date = datetime(*xlrd.xldate_as_tuple(order_date, workbook.datemode)).strftime('%Y-%m-%d')
        
        allocation_date = worksheet.cell(row, 1).value
        if allocation_date == '':
            allocation_date = None
        else:
            allocation_date = datetime(*xlrd.xldate_as_tuple(allocation_date, workbook.datemode)).strftime('%Y-%m-%d')
        
        file_row = FileFormat(
            order_date = order_date,
            allocation_date = allocation_date,
            so_number = worksheet.cell(row, 2).value,
            so_line = worksheet.cell(row, 3).value,
            cust_po = worksheet.cell(row, 4).value,
            end_user_po = worksheet.cell(row, 5).value,
            account_rep = worksheet.cell(row, 6).value,
            p_line = worksheet.cell(row, 7).value,
            td_pn = worksheet.cell(row, 8).value,
            manuf_pn = worksheet.cell(row, 9).value
        )
        
        data.append(file_row.values())
    
    df = pd.DataFrame(data, columns=FileFormat.keys()).astype(str)

    return df


In [ ]:
# Load the file into stage
put_result = session.file.put("testing.xls", "@PUBLIC.XLS_LAKE/", auto_compress = False)

In [ ]:
# Example usage:
file_path = './testing.xls'
df = read_first_workbook_from_excel(session, file_path)


In [ ]:
df

In [ ]:
session.close()